In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import mlab

from scipy import signal
import h5py, time


In [2]:
%matplotlib widget

In [3]:
import sys
sys.path.append('/home/analysis_user/New_trap_code/Tools/')
import BeadDataFile

In [4]:
##Force Calibration with single freq, with xyz2

In [5]:
Year = "2020"
Month = "03"
Day = "20"
BeadNumber =1
#folder = r"/data/new_trap/"+ Year + Month + Day
folder = r"/home/analysis_user/data_from_windows/"+ Year + Month + Day

In [ ]:
#force calibration. 
#Comparison with Nadav's-file number and location is the same one as what he provided. 

In [6]:
#response in x
ResX=np.arange(0, 0.5, 0.1)
for i in range(0,5):
    Ndata=5+i
    SigFreq=71
#load data
    fname = folder+"/Bead"+str(BeadNumber)+"/Discharge/New_Discharge"+"/Discharge_%d.h5" % Ndata
    BDF = BeadDataFile.BeadDataFile(fname)
    data = BDF.xyz2
    fsamp=5000
    res=50000
#detrend the data
    data_det=signal.detrend(data)
#Extract the PSD by matplotlib.mlab.psd
    xpsd, freqs = matplotlib.mlab.psd(data_det[0], Fs = fsamp, NFFT = res,window=mlab.window_none)
#Calculate NSD
    ResX[i]=np.sqrt(xpsd[SigFreq*10])
    print(np.sqrt(xpsd[SigFreq*10]))

No laser power data
0.0002710659109235403
No laser power data
0.0002754358247428201
No laser power data
0.00027374295606173607
No laser power data
0.00026931560399147365
No laser power data
0.0002774851026748514


In [7]:
#response in y; calculation is the same as x
ResY=np.arange(0, 0.5, 0.1)
for i in range(0,5):
    Ndata=10+i
    SigFreq=71
    fname = folder+"/Bead"+str(BeadNumber)+"/Discharge/New_Discharge"+"/Discharge_%d.h5" % Ndata
    BDF = BeadDataFile.BeadDataFile(fname)
    data = BDF.xyz2
    fsamp=5000
    res=50000
    data_det=signal.detrend(data)

    xpsd, freqs = matplotlib.mlab.psd(data_det[0], Fs = fsamp, NFFT = res,window=mlab.window_none)
    ypsd, freqs = matplotlib.mlab.psd(data_det[1], Fs = fsamp, NFFT = res,window=mlab.window_none)
    zpsd, freqs = matplotlib.mlab.psd(data_det[2], Fs = fsamp, NFFT = res,window=mlab.window_none)
    ResY[i]=np.sqrt(ypsd[SigFreq*10])    
    print(np.sqrt(ypsd[SigFreq*10]))

No laser power data
0.0003283005068077278
No laser power data
0.00033345278668207574
No laser power data
0.0003315063385773604
No laser power data
0.0003276605922930189
No laser power data
0.0003268623871889636


In [8]:
#response in z; calculation is the same as x

ResZ=np.arange(0, 0.5, 0.1)
for i in range(0,5):
    Ndata=0+i
    fname = folder+"/Bead"+str(BeadNumber)+"/Discharge/New_Discharge"+"/Discharge_%d.h5" % Ndata
    BDF = BeadDataFile.BeadDataFile(fname)
    data = BDF.xyz2
    fsamp=5000
    res=50000
    data_det=signal.detrend(data)

    xpsd, freqs = matplotlib.mlab.psd(data_det[0], Fs = fsamp, NFFT = res,window=mlab.window_none)
    ypsd, freqs = matplotlib.mlab.psd(data_det[1], Fs = fsamp, NFFT = res,window=mlab.window_none)
    zpsd, freqs = matplotlib.mlab.psd(data_det[2], Fs = fsamp, NFFT = res,window=mlab.window_none)
    ResZ[i]=np.sqrt(zpsd[SigFreq*10])    
    print(np.sqrt(zpsd[SigFreq*10]))
    

No laser power data
757.3256996396474
No laser power data
743.4378086531398
No laser power data
755.8859297272302
No laser power data
736.3401685816547
No laser power data
754.2909795107544


In [9]:
#Calculate force 
#charge is assumed to be 7e. 
q=7
V=10
F=(q*1.602e-19)*(V*2/8.6e-3*0.65)*np.sqrt(5) #this sqrt(5) is the factor to convert the force [N] to NSD [N/sqrt(Hz)] 
print(F)

3.7904472313473764e-15


In [10]:
#Calculate the conversion factor from bit to N/sqrt(Hz)
AveX=np.average(ResX)
AveY=np.average(ResY)
AveZ=np.average(ResZ)
BtoFx=F/AveX
BtoFy=F/AveY
BtoFz=F/AveZ
print(BtoFx)
print(BtoFy)
print(BtoFz)

1.3863647965894956e-11
1.1501660488405763e-11
5.0575972952157985e-18


In [11]:
##code for Transfer Function- 7Hz
#following is the code to generate transfer function with FFT

In [36]:
##X preparation
#Load file, and define relevant parameters
Ndata=11
fname = folder+"/Bead"+str(BeadNumber)+"/TransFunc"+"/TransFunc_X_m800k_150s_7hz.h5" #% Ndata
BDF = BeadDataFile.BeadDataFile(fname)
data = BDF.xyz2
tInt=150
fsamp=5000
res=Tmeas*fsamp
fBase=7

#detrend data
data_det=signal.detrend(data)
x_1000=data_det[0]
y_1000=data_det[1]
z_1000=data_det[2]
freqs=np.linspace(0,2500,(25000)+1)
    
#FFT    
xFFT=np.fft.rfft(x_1000)
yFFT=np.fft.rfft(y_1000)
zFFT=np.fft.rfft(z_1000)
#Normalization
norm = np.sqrt(2 / (res* fsamp))

#Calculate PSD from FFT
xpsd = norm**2 * (xFFT * xFFT.conj()).real
ypsd = norm**2 * (yFFT * yFFT.conj()).real
zpsd = norm**2 * (zFFT * zFFT.conj()).real

#extract phase of the transfer function
XphaseX=np.angle(xFFT)
XphaseY=np.angle(yFFT)
XphaseZ=np.angle(zFFT)

#Extract the amplitude of the transfer function
XnoiseX=np.sqrt(xpsd)
XnoiseY=np.sqrt(ypsd)
XnoiseZ=np.sqrt(zpsd)


No height data
No cantilever data
Error loading spinning data
No laser power data


In [37]:
##Y preparataion
#same calculation as x
fname = folder+"/Bead"+str(BeadNumber)+"/TransFunc"+"/TransFunc_Y_m800k_150s_7hz.h5" #% Ndata
BDF = BeadDataFile.BeadDataFile(fname)
data = BDF.xyz2 
data_det=signal.detrend(data)
x_1000=data_det[0]
y_1000=data_det[1]
z_1000=data_det[2]
freqs=np.linspace(0,2500,(25000)+1)
    
xFFT=np.fft.rfft(x_1000)
yFFT=np.fft.rfft(y_1000)
zFFT=np.fft.rfft(z_1000)
norm = np.sqrt(2 / (res* fsamp))
xpsd = norm**2 * (xFFT * xFFT.conj()).real
ypsd = norm**2 * (yFFT * yFFT.conj()).real
zpsd = norm**2 * (zFFT * zFFT.conj()).real
YphaseX=np.angle(xFFT)
YphaseY=np.angle(yFFT)
YphaseZ=np.angle(zFFT)

YnoiseX=np.sqrt(xpsd)
YnoiseY=np.sqrt(ypsd)
YnoiseZ=np.sqrt(zpsd)


No height data
No cantilever data
Error loading spinning data
No laser power data


In [38]:
##Z preparataion
#same calculation as x
fname = folder+"/Bead"+str(BeadNumber)+"/TransFunc"+"/TransFunc_Z_m800k_150s_7hz.h5" #% Ndata
BDF = BeadDataFile.BeadDataFile(fname)
data = BDF.xyz2 
data_det=signal.detrend(data)
x_1000=data_det[0]
y_1000=data_det[1]
z_1000=data_det[2]
freqs=np.linspace(0,2500,(25000)+1)
    
xFFT=np.fft.rfft(x_1000)
yFFT=np.fft.rfft(y_1000)
zFFT=np.fft.rfft(z_1000)
norm = np.sqrt(2 / (res* fsamp))
xpsd = norm**2 * (xFFT * xFFT.conj()).real
ypsd = norm**2 * (yFFT * yFFT.conj()).real
zpsd = norm**2 * (zFFT * zFFT.conj()).real
ZphaseX=np.angle(xFFT)
ZphaseY=np.angle(yFFT)
ZphaseZ=np.angle(zFFT)

ZnoiseX=np.sqrt(xpsd)
ZnoiseY=np.sqrt(ypsd)
ZnoiseZ=np.sqrt(zpsd)



No height data
No cantilever data
Error loading spinning data
No laser power data


In [15]:
#Define array for saving purpose
xPha=np.zeros(100)
yPha=np.zeros(100)
zPha=np.zeros(100)
xAmp=np.zeros(100)
yAmp=np.zeros(100)
zAmp=np.zeros(100)


In [72]:
#Remove 2pi jump in phase. 
for i in range(0,100):
    xPha[i]=XphaseX[(i+1)*tInt*fBase]-(i*i/100-round(i*i/100))*2*3.14159265+np.pi*0.5
    if(xPha[i]>3):
        xPha[i]-=2*np.pi
    yPha[i]=YphaseY[(i+1)*tInt*fBase]-(i*i/100-round(i*i/100))*2*3.14159265-np.pi*0.5
    zPha[i]=ZphaseZ[(i+1)*tInt*fBase]-(i*i/100-round(i*i/100))*2*3.14159265+np.pi*0.5

for i in range(0,99):  
    if((yPha[i+1]-yPha[i])>4):
        yPha[i+1]-=2*np.pi
    elif((yPha[i+1]-yPha[i])<-4):
        yPha[i+1]+=2*np.pi
for i in range(0,99):  
    if((yPha[i+1]-yPha[i])>4):
        yPha[i+1]-=2*np.pi
    elif((yPha[i+1]-yPha[i])<-4):
        yPha[i+1]+=2*np.pi
        
for i in range(0,99):  
    if((zPha[i+1]-zPha[i])>3):
        zPha[i+1]-=2*np.pi
    elif((zPha[i+1]-zPha[i])<-3):
        zPha[i+1]+=2*np.pi
for i in range(0,99):  
    if((zPha[i+1]-zPha[i])>4):
        zPha[i+1]-=2*np.pi
    elif((zPha[i+1]-zPha[i])<-4):
        zPha[i+1]+=2*np.pi

In [73]:
#Plot transfer function, in this case phase
freqs=np.linspace(0,2500,(375000)+1)

_,ax = plt.subplots()
plt.subplots_adjust(left=0.18, right=0.96, top=0.96, bottom=0.14)

ax.plot(np.linspace(7,700,100),xPha, '.-', label="x",markersize=1,linewidth=1)
ax.plot(np.linspace(7,700,100),yPha, '.-', label="y",markersize=1,linewidth=1)
ax.plot(np.linspace(7,700,100),zPha, '.-', label="z",markersize=1,linewidth=1)
ax.legend(fontsize=16)
ax.set_xlabel('Frequency [Hz]', fontsize=18)
ax.set_ylabel('Force NSD [N/√Hz]',fontsize=18)

ax.tick_params(axis='x', labelsize=18)
ax.tick_params(axis='y', labelsize=18)
#plt.xlim(0.1,500)
#plt.ylim(1e-7,1e-2)
plt.grid(True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [50]:
#Save phase of the transfer function
ofname = folder+"/Bead"+str(BeadNumber)+"/TransFunc"+"/TransFuncPhase_XYZ_m800k_150s_7hz.txt" 
f1 = open(ofname,"a")
for k in range(0,100):
    f1.write('%d %f %f %f\n' % (k+1, xPha[k], yPha[k], zPha[k]))
f1.close()

In [51]:
#phase data with 1 Hz trans func data

In [20]:
##X preparation
#Load file, and define relevant parameters
Ndata=11
fname = folder+"/Bead"+str(BeadNumber)+"/TransFunc"+"/Repeat_TransFunc_20200104/TransFunc_Z_m300k_250s_1hz.h5" #% Ndata
BDF = BeadDataFile.BeadDataFile(fname)
data = BDF.xyz2
tInt=250
fsamp=5000
res=tInt*fsamp
fBase=1

#detrend data
data_det=signal.detrend(data)
x_1000=data_det[0]
y_1000=data_det[1]
z_1000=data_det[2]
freqs=np.linspace(0,2500,(25000)+1)
    
#FFT
xFFT=np.fft.rfft(x_1000)
yFFT=np.fft.rfft(y_1000)
zFFT=np.fft.rfft(z_1000)
#Normalization
norm = np.sqrt(2 / (res* fsamp))

#Calculate PSF from FFT
xpsd = norm**2 * (xFFT * xFFT.conj()).real
ypsd = norm**2 * (yFFT * yFFT.conj()).real
zpsd = norm**2 * (zFFT * zFFT.conj()).real

#extract phase of the transfer function
XphaseX=np.angle(xFFT)
XphaseY=np.angle(yFFT)
XphaseZ=np.angle(zFFT)

#Extract the amplitude of the transfer function
XnoiseX=np.sqrt(xpsd)
XnoiseY=np.sqrt(ypsd)
XnoiseZ=np.sqrt(zpsd)

No height data
No cantilever data
Error loading spinning data
No laser power data


In [21]:
##Y preparataion
#same calculation as x
fname = folder+"/Bead"+str(BeadNumber)+"/TransFunc"+"/Repeat_TransFunc_20200104/TransFunc_Z_m300k_250s_1hz.h5" #% Ndata
BDF = BeadDataFile.BeadDataFile(fname)
data = BDF.xyz2 
data_det=signal.detrend(data)
x_1000=data_det[0]
y_1000=data_det[1]
z_1000=data_det[2]
freqs=np.linspace(0,2500,(25000)+1)
    
xFFT=np.fft.rfft(x_1000)
yFFT=np.fft.rfft(y_1000)
zFFT=np.fft.rfft(z_1000)
norm = np.sqrt(2 / (res* fsamp))
xpsd = norm**2 * (xFFT * xFFT.conj()).real
ypsd = norm**2 * (yFFT * yFFT.conj()).real
zpsd = norm**2 * (zFFT * zFFT.conj()).real
YphaseX=np.angle(xFFT)
YphaseY=np.angle(yFFT)
YphaseZ=np.angle(zFFT)

YnoiseX=np.sqrt(xpsd)
YnoiseY=np.sqrt(ypsd)
YnoiseZ=np.sqrt(zpsd)


No height data
No cantilever data
Error loading spinning data
No laser power data


In [22]:
##Z preparataion
#same calculation as x
fname = folder+"/Bead"+str(BeadNumber)+"/TransFunc"+"/Repeat_TransFunc_20200104/TransFunc_Z_m300k_250s_1hz.h5" #% Ndata
BDF = BeadDataFile.BeadDataFile(fname)
data = BDF.xyz2 
data_det=signal.detrend(data)
x_1000=data_det[0]
y_1000=data_det[1]
z_1000=data_det[2]
freqs=np.linspace(0,2500,(25000)+1)
    
xFFT=np.fft.rfft(x_1000)
yFFT=np.fft.rfft(y_1000)
zFFT=np.fft.rfft(z_1000)
norm = np.sqrt(2 / (res* fsamp))
xpsd = norm**2 * (xFFT * xFFT.conj()).real
ypsd = norm**2 * (yFFT * yFFT.conj()).real
zpsd = norm**2 * (zFFT * zFFT.conj()).real
ZphaseX=np.angle(xFFT)
ZphaseY=np.angle(yFFT)
ZphaseZ=np.angle(zFFT)

ZnoiseX=np.sqrt(xpsd)
ZnoiseY=np.sqrt(ypsd)
ZnoiseZ=np.sqrt(zpsd)

No height data
No cantilever data
Error loading spinning data
No laser power data


In [25]:
#Remove 2pi jump in phase. 
for i in range(0,100):
    xPha[i]=XphaseX[(i+1)*tInt*fBase]-(i*i/100-round(i*i/100))*2*3.14159265-np.pi*0.5
    xAmp[i]=XnoiseX[i]
    if(xPha[i]<-3):
        xPha[i]+=2*np.pi
    yPha[i]=YphaseY[(i+1)*tInt*fBase]-(i*i/100-round(i*i/100))*2*3.14159265+np.pi*0.5
    yAmp[i]=YnoiseY[i]
    if(yPha[i]>3):
        yPha[i]-=2*np.pi
    zPha[i]=ZphaseZ[(i+1)*tInt*fBase]-(i*i/100-round(i*i/100))*2*3.14159265+np.pi*0.5
    zAmp[i]=ZnoiseZ[i]

    
        
for i in range(0,99):  
    if((zPha[i+1]-zPha[i])>4):
        zPha[i+1]-=2*np.pi
    elif((zPha[i+1]-zPha[i])<-4):
        zPha[i+1]+=2*np.pi
for i in range(0,99):  
    if((zPha[i+1]-zPha[i])>4):
        zPha[i+1]-=2*np.pi
    elif((zPha[i+1]-zPha[i])<-4):
        zPha[i+1]+=2*np.pi

In [26]:
#Plot transfer function phase
freqs=np.linspace(0,2500,(375000)+1)

_,ax = plt.subplots()
plt.subplots_adjust(left=0.18, right=0.96, top=0.96, bottom=0.14)

ax.semilogx(np.linspace(1,100,100),xPha, '.-', label="x",markersize=1,linewidth=1)
ax.semilogx(np.linspace(1,100,100),yPha, '.-', label="y",markersize=1,linewidth=1)
ax.semilogx(np.linspace(1,100,100),zPha, '.-', label="z",markersize=1,linewidth=1)
ax.legend(fontsize=16)
ax.set_xlabel('Phase [rad]', fontsize=18)
ax.set_ylabel('Force NSD [N/√Hz]',fontsize=18)

ax.tick_params(axis='x', labelsize=18)
ax.tick_params(axis='y', labelsize=18)
#plt.xlim(0.1,500)
#plt.ylim(1e-7,1e-2)
plt.grid(True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [43]:
#Plot transfer function amplitude
freqs=np.linspace(0,2500,(375000)+1)

_,ax = plt.subplots()
plt.subplots_adjust(left=0.18, right=0.96, top=0.96, bottom=0.14)

#ax.plot(np.linspace(1,100,100),xAmp, '.-', label="x",markersize=1,linewidth=1)
#ax.plot(np.linspace(1,100,100),yAmp, '.-', label="y",markersize=1,linewidth=1)
ax.plot(np.linspace(1,100,100),zAmp, '.-', label="z",markersize=1,linewidth=1)
ax.legend(fontsize=16)
ax.set_xlabel('Frequency [Hz]', fontsize=18)
ax.set_ylabel('Force NSD [N/√Hz]',fontsize=18)

ax.tick_params(axis='x', labelsize=18)
ax.tick_params(axis='y', labelsize=18)
#plt.xlim(0.1,500)
#plt.ylim(1e-7,1e-2)
plt.grid(True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [34]:
#save transfer function, in this case phase
ofname = folder+"/Bead"+str(BeadNumber)+"/TransFunc"+"/RepeatTransFuncPhase_XYZ_m300k_250s_1hz_2.txt"
f1 = open(ofname,"a")
for k in range(0,100):
    f1.write('%d %f %f %f\n' % (k+1, xPha[k], yPha[k], zPha[k]))
f1.close()

In [16]:
#preparation to save full amplitude information
#Here, there is no conversion factor
q=20
V=0.65
F=(q*1.602e-19)*(V*2/8.6e-3*0.65)
Nharm=100


FnoiseXX=XnoiseX
FnoiseXY=XnoiseY
FnoiseXZ=XnoiseZ
FnoiseYX=YnoiseX
FnoiseYY=YnoiseY
FnoiseYZ=YnoiseZ
FnoiseZX=ZnoiseX
FnoiseZY=ZnoiseY
FnoiseZZ=ZnoiseZ

TFXX=np.zeros(Nharm)
TFXY=np.zeros(Nharm)
TFXZ=np.zeros(Nharm)
TFYX=np.zeros(Nharm)
TFYY=np.zeros(Nharm)
TFYZ=np.zeros(Nharm)
TFZX=np.zeros(Nharm)
TFZY=np.zeros(Nharm)
TFZZ=np.zeros(Nharm)

FreqTF=np.zeros(Nharm)
for i in range(Nharm):
    FreqTF[i]=fbase*(i+1)
    TFXX[i]=FnoiseXX[fbase*Tmeas*(i+1)]
    TFXY[i]=FnoiseXY[fbase*Tmeas*(i+1)]
    TFXZ[i]=FnoiseXZ[fbase*Tmeas*(i+1)]
    TFYX[i]=FnoiseYX[fbase*Tmeas*(i+1)]
    TFYY[i]=FnoiseYY[fbase*Tmeas*(i+1)]
    TFYZ[i]=FnoiseYZ[fbase*Tmeas*(i+1)]
    TFZX[i]=FnoiseZX[fbase*Tmeas*(i+1)]
    TFZY[i]=FnoiseZY[fbase*Tmeas*(i+1)]
    TFZZ[i]=FnoiseZZ[fbase*Tmeas*(i+1)]

In [23]:
#save full amplitude information
ofname = folder+"/Bead"+str(BeadNumber)+"/TransFunc"+"/TFBead%d_Bit.txt" % (Ndata)

f1 = open(ofname,"a")
for i in range(Nharm):
    f1.write('%f %e %e %e %e %e %e %e %e %e\n' % (FreqTF[i],TFXX[i],TFXY[i],TFXZ[i],
                                                  TFYX[i],TFYY[i],TFYZ[i],TFZX[i],TFZY[i],TFZZ[i]))
f1.close()

In [26]:
##combining the "#analyzing multiple data in single number,"  
##"##for Noise only analysis for sensitivity. Used bin of 3*n+0.7 Hz, Nint=0", and SortDataTo Harmonics

In [37]:
import scipy, sys, time

In [38]:
#Gravity Measurement

In [41]:
#Harmonics data extraction for the gravity data
#FIle name is somehow different from the data we use for analysis, but the way it is analyzed is the same. 
import time
start_time = time.time()

#DefineParameters
Ndata=1
Dx=340
Fshake=3
DataN=2500
HarmN=20
Noff=3
ArraySize=(DataN,HarmN)
DataSpaceXSigAmp=np.zeros(ArraySize)
DataSpaceYSigAmp=np.zeros(ArraySize)
DataSpaceZSigAmp=np.zeros(ArraySize)
DataSpaceXSigPha=np.zeros(ArraySize)
DataSpaceYSigPha=np.zeros(ArraySize)
DataSpaceZSigPha=np.zeros(ArraySize)

DataSpaceXNoiAmp=np.zeros(ArraySize)
DataSpaceYNoiAmp=np.zeros(ArraySize)
DataSpaceZNoiAmp=np.zeros(ArraySize)
DataSpaceXNoiPha=np.zeros(ArraySize)
DataSpaceYNoiPha=np.zeros(ArraySize)
DataSpaceZNoiPha=np.zeros(ArraySize)

for k in range(0,DataN):
##load the data
    NSubData=k
    fname = folder+"/Bead"+str(BeadNumber)+"/Shaking"+"/Shaking%d/NoSpinShaking%d_%d.h5" % (Dx,Ndata,NSubData)
    BDF = BeadDataFile.BeadDataFile(fname)
    data = BDF.xyz2
    fsamp=5000
    res=50000

#detrend data
    data_det=signal.detrend(data)
    x_1000=data_det[0]
    y_1000=data_det[1]
    z_1000=data_det[2]
    freqs=np.linspace(0,2500,(25000)+1)
    
    
#FFT   
    xFFT=np.fft.rfft(x_1000)
    yFFT=np.fft.rfft(y_1000)
    zFFT=np.fft.rfft(z_1000)
#normalization
    norm = np.sqrt(2 / (res* fsamp))
#PSD calculation from FFT
    xpsd = norm**2 * (xFFT * xFFT.conj()).real
    ypsd = norm**2 * (yFFT * yFFT.conj()).real
    zpsd = norm**2 * (zFFT * zFFT.conj()).real
  
    f0=71
#NSD calculation, together with bit to N/sqrt(Hz) conversion. Ignore the commented out part. 
    xnsd=np.sqrt(xpsd)*BtoFx#*F0x*np.sqrt((freqs*freqs-w0x*w0x)*(freqs*freqs-w0x*w0x)+gx*gx*freqs*freqs)
    ynsd=np.sqrt(ypsd)*BtoFy#*F0y*np.sqrt((freqs*freqs-w0y*w0y)*(freqs*freqs-w0y*w0y)+gy*gy*freqs*freqs)
    znsd=np.sqrt(zpsd)*BtoFz#*F0z*np.sqrt((f0*f0-w0z*w0z)*(f0*f0-w0z*w0z)+gz*gz*f0*f0)

#implement the effect of transfer function to the amplitude, and extract phase (here phase does have trans. func)
#The effect of the trans. func. of the phase is implemented in the signal model. 
for i in range(1,(HarmN+1)):
        DataSpaceXSigAmp[k][i-1]=xnsd[Fshake*10*i]/XtfX[Fshake*10*i-10]*XtfX[f0*10-10]
        DataSpaceXSigPha[k][i-1]=np.angle(xFFT[Fshake*10*i]) 
        DataSpaceYSigAmp[k][i-1]=ynsd[Fshake*10*i]/YtfY[Fshake*10*i-10]*YtfY[f0*10-10]
        DataSpaceYSigPha[k][i-1]=np.angle(yFFT[Fshake*10*i]) 
        DataSpaceZSigAmp[k][i-1]=znsd[Fshake*10*i]/ZtfZ[Fshake*10*i-10]*ZtfZ[f0*10-10]
        DataSpaceZSigPha[k][i-1]=np.angle(zFFT[Fshake*10*i]) 
        
##save noise bin data to the array
    for i in range(1,(HarmN+1)):
        DataSpaceXNoiAmp[k][i-1]=xnsd[Fshake*10*i-Noff]/XtfX[Fshake*10*i-10]*XtfX[f0*10-10]
        DataSpaceXNoiPha[k][i-1]=np.angle(xFFT[Fshake*10*i-Noff]) 
        DataSpaceYNoiAmp[k][i-1]=ynsd[Fshake*10*i-Noff]/YtfY[Fshake*10*i-10]*YtfY[f0*10-10]
        DataSpaceYNoiPha[k][i-1]=np.angle(yFFT[Fshake*10*i-Noff]) 
        DataSpaceZNoiAmp[k][i-1]=znsd[Fshake*10*i-Noff]/ZtfZ[Fshake*10*i-10]*ZtfZ[f0*10-10]
        DataSpaceZNoiPha[k][i-1]=np.angle(zFFT[Fshake*10*i-Noff]) 
  
    if (k%100)==0:
        print(k)
        
##write the harmonics data
for i in range(1,(HarmN+1)):
    ofname = folder+"/Bead"+str(BeadNumber)+"/Shaking"+"/Shaking%d/SignalNoSpinShaking%d_%dthHarm.txt" % (Dx,Ndata,i)
    f1 = open(ofname,"a")
    for k in range(0,DataN):
        f1.write('%d %e %e %e %f %f %f\n' % (k, DataSpaceXSigAmp[k][i-1], DataSpaceYSigAmp[k][i-1], DataSpaceZSigAmp[k][i-1], 
                                            DataSpaceXSigPha[k][i-1], DataSpaceYSigPha[k][i-1], DataSpaceZSigPha[k][i-1]))
    f1.close()
    ofname = folder+"/Bead"+str(BeadNumber)+"/Shaking"+"/Shaking%d/NoiseNoSpinShaking%d_%dthHarm.txt" % (Dx,Ndata,i)
    f1 = open(ofname,"a")
    for k in range(0,DataN):
        f1.write('%d %e %e %e %f %f %f\n' % (k, DataSpaceXNoiAmp[k][i-1], DataSpaceYNoiAmp[k][i-1], DataSpaceZNoiAmp[k][i-1], 
                                            DataSpaceXNoiPha[k][i-1], DataSpaceYNoiPha[k][i-1], DataSpaceZNoiPha[k][i-1]))
    f1.close()
    
print ("Total run time is ", time.time() - start_time, " s")

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
Total run time is  41.755202293395996  s


In [ ]:
##########Following is the signal model generation##########

In [ ]:
##this is derived from Signal_Model_Usage_Example_20191127.ipynb
#Loading necessary files for signal model calculation

import numpy as np
import pickle as pkl
import scipy.interpolate as interp
import scipy, sys, time
from bisect import bisect_left
import matplotlib.pyplot as plt
sys.path.append('/home/analysis_user/New_trap_code/Tools/')
import BeadDataFile
from discharge_tools import load_dir
sys.path.append('/home/analysis_user/New_trap_code/SensitivityFramework/')
from signal_model_utilities_v3 import *



In [ ]:
#for z force, 3/20 Bead1, Shaking378_3
#preparation of parameters. Here, I calculate signal model for 70 different lambdas. 
for i in range(0,70):
    L=pow(10,i/30.0-0.3)
    D=10.39+3.8 #this becomes14.19
    H=-15.7
    Fshake=3
#calculate 1 second lont sitnal data
    test = force_vs_time(D,H,200,Fshake,"z",L,yuk_or_grav="yuk",alpha=1e10,offset_y=4.9,bead_size=7.6)

#define output file and some other parameters. 
    ofname = "/data/new_trap/Results/AkioK/2020_11_18_Analysis0320Bead1/SignalZD%dumH%dumL%.02f.txt" % (D,H,L)
    fsamp=5000
    res=50000

#normalization for FFT    
    norm = np.sqrt(2 / (res* fsamp))

#definition of an array to store the signal in time domain.     
    s = np.arange(0, 5, 0.0001)

#by repeating 1 s data for 10 times, 10 s long data is generated, so that it can be FFTed in the same way as the 
#experimental data. 
    for i in range(0,10):
        for j in range(0,5000):
            s[j+i*5000]=test[1][j]
    
#FFT-> PSD calculation->normalization->NSD calculation    
    sFFT=np.fft.rfft(s)
    spsd = (sFFT * sFFT.conj()).real
    spsd = norm**2 * (sFFT * sFFT.conj()).real
    snsd=np.sqrt(spsd)
    
#save signal model    
    f1 = open(ofname,"a")
    
    for i in range(1,101):
        Ints=sFFT[Fshake*10*i]
        IntNSDs=snsd[Fshake*10*i]
     
        SigAmpX=IntNSDs
        SigPhaX=np.angle(Ints)         
        f1.write('%f %e %e %e %e %e %e\n' % (Fshake*i, SigAmpX, 0,0, SigPhaX, 0,0))
    f1.close()